# Hyperparameter Tunning Result 3
Andrew E. Davidson  
aedavids@ucsc.edu  
1/3/23


pivot see Hyperparameter Tunning Result 4. 
best500FindAllDegree1_wl500 has not completed as of of 1/8 11 am. Exploration of best500FindAllDegree1_wl500 Intersectiron Dict suggest a better approach. We choose top n={1,3,5,...} of the degree1 genes for each category

TODO

### Pipeline overview

- stage 1: best200GTEx_TCGA
  + selected the top 200 biologically signifigant genes. There are lots of high degree intersections, i.e. poor discriminators. 

- stage 2: aedwip
  + ensure all categories have at least 1 degree 1 gene
    
- stage 3: best200Degree1GTEx_TCGA.sh
  + drops all genes that are not in degree 1 intersections. 

In [1]:
import ast
import ipynbname

# use display() to print an html version of a data frame
# useful if dataFrame output is not generated by last like of cell
from IPython.display import display

import numpy as np
import pandas as pd
# display all columns
pd.set_option('display.max_columns', None)

import pathlib as pl
import pprint as pp
import os
import sys

In [2]:
# setting the python path allows us to run python scripts from using
# the CLI. 
ORIG_PYTHONPATH = os.environ['PYTHONPATH']

notebookPath = ipynbname.path()
deconvolutionModules = notebookPath.parent.joinpath("../../python")
print("deconvolutionModules: {}\n".format(deconvolutionModules))

PYTHONPATH = ORIG_PYTHONPATH + f':{deconvolutionModules}'
print("PYTHONPATH: {}\n".format(PYTHONPATH))

os.environ["PYTHONPATH"] = PYTHONPATH
PYTHONPATH = os.environ["PYTHONPATH"]
print("PYTHONPATH: {}\n".format(PYTHONPATH))

deconvolutionModules: /private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python

PYTHONPATH: :/private/home/aedavids/extraCellularRNA/src:/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python



In [3]:
# to be able to import our local python files we need to set the sys.path
# https://stackoverflow.com/a/50155834
sys.path.append( str(deconvolutionModules) )
print("\nsys.path:\n{}\n".format(sys.path))


sys.path:
['/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning', '/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning', '/private/home/aedavids/extraCellularRNA/src', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python311.zip', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/lib-dynload', '', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages/setuptools-57.4.0-py3.9.egg', '/private/home/aedavids/miniconda3/envs/extraCellularRNA/lib/python3.11/site-packages/pip-21.1.3-py3.9.egg', '/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/jupyterNotebooks/hyperParameterTunning/../../python']



In [4]:
from analysis.hyperParameterTunningMetrics import metricsRunner, elifeCols, lungCols
from analysis.hyperParameterTunningMetrics import findSummaryMetricsCols
from analysis.utilities import findAllCategories, findAllGenes
from analysis.utilities import findIntersectionsWithDegree
from analysis.utilities import loadDictionary

In [5]:
root = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category"
notebookName = ipynbname.name()
outDir = f'{root}/hyperParameter/{notebookName}.out'
print( f'output dir: \n{outDir}' )
os.makedirs(outDir, exist_ok=True)

output dir: 
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/hyperParameter/hyperparameterTunningResults3.out


## stage 1: Best N 
selected top N genes 

In [6]:
def best(threshold):
    '''
    wrapped to avoid global polution
    '''
    bestResultsDirs = [    
            "best20GTEx_TCGA",     
            "best25GTEx_TCGA",     
            "best30GTEx_TCGA",     
            "best50GTEx_TCGA",     
            "best100GTEx_TCGA",     
            "best200GTEx_TCGA",  
            "best500GTEx_TCGA",     
    ]
    
    outFilePrefix = "best"
    metric = 'sensitivity'    
    bestDF, bestBellowThresholdDF = metricsRunner(root, outDir, outFilePrefix, bestResultsDirs, 
                           metric=metric, threshold=threshold, verbose=False)
    
    display( bestDF.loc[:, findSummaryMetricsCols('sensitivity') + elifeCols  ] )
    
    
    stageName =  "best500GTEx_TCGA"
    print(f'\n {stageName} classes < {threshold} {metric}')
    selectRowsBest500Bellow = bestBellowThresholdDF.loc[:, "stage"] == stageName
    
    display( bestBellowThresholdDF.loc[selectRowsBest500Bellow, 'category'] )


threshold = 0.7
best(threshold)


best20GTEx_TCGA types without degree 1 intersections: 
 {'READ', 'Brain_Spinal_cord_cervical_c-1', 'Brain_Cortex', 'Skin_Sun_Exposed_Lower_leg', 'SKCM', 'KIRC', 'Adrenal_Gland', 'Brain_Hippocampus', 'KICH', 'Artery_Aorta', 'Ovary', 'HNSC', 'Brain_Substantia_nigra', 'Colon_Transverse', 'KIRP', 'Esophagus_Muscularis', 'Brain_Cerebellar_Hemisphere', 'Skin_Not_Sun_Exposed_Suprapubic', 'Testis', 'Colon_Sigmoid', 'Nerve_Tibial', 'Brain_Anterior_cingulate_cortex_BA24', 'Adipose_Subcutaneous', 'Cells_EBV-transformed_lymphocytes', 'Artery_Tibial', 'Brain_Nucleus_accumbens_basal_ganglia', 'Esophagus_Mucosa', 'Cells_Cultured_fibroblasts', 'Uterus', 'THYM', 'Brain_Caudate_basal_ganglia', 'PRAD', 'UCS', 'COAD', 'Brain_Frontal_Cortex_BA9', 'Brain_Amygdala', 'LGG', 'ACC', 'Heart_Left_Ventricle', 'Liver', 'Brain_Putamen_basal_ganglia', 'Brain_Hypothalamus', 'Spleen', 'Brain_Cerebellum', 'CESC'}

best25GTEx_TCGA types without degree 1 intersections: 
 {'READ', 'Brain_Spinal_cord_cervical_c-1', 'Brain_

id,mean_sensitivity,std_sensitivity,numGenes,numTypes,numDegree1,numAboveThreshold,Lung,LUAD,LUSC,Colon_Sigmoid,Colon_Transverse,COAD,READ,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,ESCA,Liver,LIHC,Stomach,STAD
best20GTEx_TCGA,0.704458,0.216465,258,83,38,45,0.971,0.447,0.495,0.902,0.539,0.437,0.821,0.458,0.895,0.605,0.189,0.934,0.713,0.702,0.276
best25GTEx_TCGA,0.709084,0.216436,322,83,44,43,0.988,0.440,0.508,0.906,0.539,0.456,0.804,0.516,0.895,0.612,0.180,0.934,0.717,0.698,0.271
best30GTEx_TCGA,0.711289,0.215893,382,83,45,47,0.988,0.469,0.505,0.911,0.543,0.475,0.786,0.529,0.913,0.634,0.135,0.897,0.780,0.702,0.271
best50GTEx_TCGA,0.722566,0.208438,597,83,49,49,0.974,0.469,0.571,0.911,0.531,0.532,0.786,0.556,0.925,0.686,0.171,0.875,0.789,0.688,0.244
best100GTEx_TCGA,0.734277,0.206544,1149,83,62,50,0.988,0.528,0.654,0.920,0.547,0.570,0.732,0.600,0.934,0.718,0.117,0.794,0.789,0.698,0.204
best200GTEx_TCGA,0.751120,0.197365,2217,83,73,53,0.988,0.605,0.708,0.911,0.551,0.563,0.714,0.627,0.940,0.751,0.135,0.912,0.812,0.726,0.244
best500GTEx_TCGA,0.767325,0.195702,5140,83,77,55,0.997,0.670,0.731,0.924,0.551,0.576,0.732,0.631,0.940,0.777,0.162,0.949,0.816,0.735,0.231



 best500GTEx_TCGA classes < 0.7 sensitivity


id
0                                       ACC
7                                      BLCA
8                                      BRCA
10                           Brain_Amygdala
11     Brain_Anterior_cingulate_cortex_BA24
12              Brain_Caudate_basal_ganglia
16                 Brain_Frontal_Cortex_BA9
17                        Brain_Hippocampus
18                       Brain_Hypothalamus
19    Brain_Nucleus_accumbens_basal_ganglia
20              Brain_Putamen_basal_ganglia
23                    Breast_Mammary_Tissue
26                                     COAD
31                         Colon_Transverse
33                                     ESCA
34      Esophagus_Gastroesophageal_Junction
38                                     HNSC
41                                     KICH
42                                     KIRC
43                                     KIRP
45                                      LGG
47                                     LUAD
55                           

## Explore best200 and best500
Does this provide support evidence algo designed on 1/2/24 is worth pursuing?

In [7]:
def exploreDegree1Interesections(intesectionPath : str):
    intersectionDict = loadDictionary(intesectionPath)
    degree1Dict = findIntersectionsWithDegree(intersectionDict, degree=1)
    totalUniqueGenes = 0
    for category,genes in degree1Dict.items():
        nUnique = len(genes)
        totalUniqueGenes += nUnique
        print( f'category: {category} : num genes : {nUnique}' )
    
    print( f'\n totalUniqueGenes : {totalUniqueGenes}' )

    return (intersectionDict, degree1Dict, totalUniqueGenes)
 

In [8]:
# use best20 as a baseline
def exploreBest20():
    best20DictPathDictPath = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best20GTEx_TCGA/training/best20GTEx_TCGA.sh.out/upsetPlot.out/best20.intersection.dict"
    intersectionDict, degree1Dict, totalUniqueGenes = exploreDegree1Interesections(best20DictPathDictPath)

exploreBest20()

category: ('Adipose_Visceral_Omentum',) : num genes : 1
category: ('Artery_Coronary',) : num genes : 2
category: ('BLCA',) : num genes : 4
category: ('BRCA',) : num genes : 3
category: ('Bladder',) : num genes : 6
category: ('Breast_Mammary_Tissue',) : num genes : 1
category: ('CHOL',) : num genes : 2
category: ('Cervix_Endocervix',) : num genes : 6
category: ('DLBC',) : num genes : 3
category: ('ESCA',) : num genes : 1
category: ('Esophagus_Gastroesophageal_Junction',) : num genes : 1
category: ('GBM',) : num genes : 3
category: ('Heart_Atrial_Appendage',) : num genes : 3
category: ('Kidney_Cortex',) : num genes : 1
category: ('LIHC',) : num genes : 1
category: ('LUAD',) : num genes : 2
category: ('LUSC',) : num genes : 1
category: ('Lung',) : num genes : 1
category: ('MESO',) : num genes : 2
category: ('Minor_Salivary_Gland',) : num genes : 1
category: ('Muscle_Skeletal',) : num genes : 3
category: ('OV',) : num genes : 1
category: ('PAAD',) : num genes : 7
category: ('PCPG',) : num 

In [9]:
def exploreBest200():
    best200DictPathDictPath = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best200GTEx_TCGA/training/best200GTEx_TCGA.sh.out/upsetPlot.out/best200.intersection.dict"
    intersectionDict, degree1Dict, totalUniqueGenes = exploreDegree1Interesections(best200DictPathDictPath)
    return (intersectionDict, degree1Dict, totalUniqueGenes)
    
bet200IntersectionDict, best200Degree1Dict, best200TotalUniqueGenes = exploreBest200()

category: ('ACC',) : num genes : 7
category: ('Adipose_Subcutaneous',) : num genes : 1
category: ('Adipose_Visceral_Omentum',) : num genes : 2
category: ('Adrenal_Gland',) : num genes : 4
category: ('Artery_Aorta',) : num genes : 1
category: ('Artery_Coronary',) : num genes : 4
category: ('Artery_Tibial',) : num genes : 3
category: ('BLCA',) : num genes : 13
category: ('BRCA',) : num genes : 15
category: ('Bladder',) : num genes : 32
category: ('Brain_Cerebellar_Hemisphere',) : num genes : 2
category: ('Brain_Cerebellum',) : num genes : 1
category: ('Brain_Cortex',) : num genes : 1
category: ('Brain_Frontal_Cortex_BA9',) : num genes : 2
category: ('Brain_Spinal_cord_cervical_c-1',) : num genes : 1
category: ('CESC',) : num genes : 10
category: ('CHOL',) : num genes : 20
category: ('COAD',) : num genes : 4
category: ('Cells_Cultured_fibroblasts',) : num genes : 5
category: ('Cells_EBV-transformed_lymphocytes',) : num genes : 1
category: ('Cervix_Endocervix',) : num genes : 96
category: 

### We found 22 unique LUAD and 19 unique LUSC
Best 200 returns at most 200 deseq results rows. These rows meet our biologic signigance criteria. Over all sensitivity is not great. Keep in mind best200 has 2059 many of these are poor discriminators. ie they are intersections with high a high degree

In [10]:
best2001vsAllResultsDir = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best200GTEx_TCGA/training/best200GTEx_TCGA.sh.out/GTEx_TCGA-design-tilda_gender_category-padj-0001-lfc-20-n-200"
LUADPath = f'{best2001vsAllResultsDir}/LUAD_vs_all.results'
LUADResultsDF = pd.read_csv(LUADPath)

print(f'LUADResultsDF.shape : {LUADResultsDF.shape}')
# LUADResultsDF.head()
best200Genes = best200Degree1Dict[('LUAD',)]
selectRows = LUADResultsDF.loc[:, "name"].isin(best200Genes)
LUADResultsDF.loc[selectRows, :]

LUADResultsDF.shape : (200, 7)


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
96,CPM,2031.120136,2.243723,0.091136,24.619449,7.820894e-134,1.617437e-131
103,NAPSA,1863.020676,6.808249,0.160603,42.391826,0.000000e+00,0.000000e+00
113,SEMA6A,1659.715353,-2.506262,0.085304,-29.380225,9.826587e-190,6.130019e-187
129,HMGB3,1429.246162,2.026619,0.073351,27.629207,4.961875e-168,2.195514e-165
139,CRLF1,1287.980457,3.044996,0.116143,26.217546,1.676690e-151,5.023864e-149
149,SCN7A,1214.557916,2.007483,0.139291,14.412177,4.338181e-47,9.710635e-46
156,ADCY2,1143.054913,-3.372049,0.141744,-23.789673,4.271969e-125,7.322577e-123
160,DRAM1,1101.891449,2.000329,0.069934,28.603167,6.136155e-180,3.288720e-177
161,KRT23,1100.808127,-2.130086,0.190668,-11.171684,5.610699e-29,5.753272e-28
162,AC006115.2,1088.317661,-3.549901,0.154433,-22.986742,6.326273e-117,9.084289e-115


In [11]:
LUSCPath = f'{best2001vsAllResultsDir}/LUSC_vs_all.results'
LUSCResultsDF = pd.read_csv(LUSCPath)

print(f'LUSCResultsDF.shape : {LUSCResultsDF.shape}')
best200Genes = best200Degree1Dict[('LUSC',)]
selectRows = LUSCResultsDF.loc[:, "name"].isin(best200Genes)
LUSCResultsDF.loc[selectRows, :]

LUSCResultsDF.shape : (200, 7)


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
89,SRXN1,2153.961341,2.216050,0.065354,33.908658,4.965815e-252,5.671102e-249
100,ACKR3,1956.341452,2.100492,0.087331,24.052191,7.918539e-128,1.030983e-125
101,GCLC,1919.390917,2.091534,0.056606,36.948753,7.627089e-299,2.540520e-295
131,RNF157,1485.068661,-2.174760,0.117775,-18.465401,3.921031e-76,1.471620e-74
147,RBPMS2,1295.946716,-2.145042,0.118793,-18.057046,6.944102e-73,2.386610e-71
154,QPRT,1255.637483,-2.049667,0.106324,-19.277617,8.280377e-83,3.677499e-81
158,REEP6,1224.747370,-2.717801,0.123550,-21.997622,3.034756e-107,2.511434e-105
163,NEGR1,1178.589947,-2.041367,0.102591,-19.898130,4.223964e-88,2.175723e-86
164,RORC,1176.097480,-2.607522,0.143041,-18.229147,3.030089e-74,1.078501e-72
173,ME1,1133.241584,2.036512,0.077647,26.227782,1.281489e-151,2.799039e-149


# stage 2: aedwip find all degree1

Expect best25GTEx_TCGA and best20FindAllDegree1_wl5 to have same degree1. Best25 should have more genes. actually did a little better. 

b20FindAll has 28 more genes than best20 and 22 less genes then best25.

b20FindAll has 8 more categories with degree1 genes than b20 and has 2 more degree1 categories and b25

best20 and b20Findall have the same number of categories. this is 2 more than best25!

In [12]:
def bestAllDegree1(threshold):
    '''
    wrapped to avoid global polution
    '''
    bestResultsDirs = [    
        "best20GTEx_TCGA",     
        "best25GTEx_TCGA",
        "best20FindAllDegree1_wl5",     
              
    ]

    
    outFilePrefix = "bestAllDegree1"
    metric = 'sensitivity'    
    bestDF, bestBellowThresholdDF = metricsRunner(root, outDir, outFilePrefix, bestResultsDirs, 
                           metric=metric, threshold=threshold, verbose=True)

    print(f'\n******\nwe expect best25GTEx_TCGA and best20FindAllDegree1_wl5 to have same degree1. best25 should have more genes\n')
    
    display( bestDF.loc[:, findSummaryMetricsCols('sensitivity') + elifeCols  ] )
    
    
    stageName =  "best20FindAllDegree1_wl5"
    print(f'\n {stageName} classes < {threshold} {metric}')
    selectRowsBellow = bestBellowThresholdDF.loc[:, "stage"] == stageName
    
    display( bestBellowThresholdDF.loc[selectRowsBellow, 'category'] )


bestAllDegree1(threshold)

path : /private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best20GTEx_TCGA

load best20GTEx_TCGA :
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best20GTEx_TCGA/training/best20GTEx_TCGA.sh.out/metrics/metricsRounded.csv

load
/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best20GTEx_TCGA/training/best20GTEx_TCGA.sh.out/upsetPlot.out/best20.intersection.dict

best20GTEx_TCGA types without degree 1 intersections: 
 {'READ', 'Brain_Spinal_cord_cervical_c-1', 'Brain_Cortex', 'Skin_Sun_Exposed_Lower_leg', 'SKCM', 'KIRC', 'Adrenal_Gland', 'Brain_Hippocampus', 'KICH', 'Artery_Aorta', 'Ovary', 'HNSC', 'Brain_Substantia_nigra', 'Colon_Transverse', 'KIRP', 'Esophagus_Muscularis', 'Brain_Cerebellar_Hemisphere', 'Skin_Not_Sun_Exposed_Suprapubic', 'Testis', 'Colon_Sigmoid', 'Nerve_Tibial', 'Brain_Anterior_cingulate_cortex_BA24', 'Adipose_Subcutaneous', 'Cells_EBV-transformed_lymphocytes', 'Artery_Tibial', 'Brain_Nucleus_accumbens_basa

id,mean_sensitivity,std_sensitivity,numGenes,numTypes,numDegree1,numAboveThreshold,Lung,LUAD,LUSC,Colon_Sigmoid,Colon_Transverse,COAD,READ,Esophagus_Gastroesophageal_Junction,Esophagus_Mucosa,Esophagus_Muscularis,ESCA,Liver,LIHC,Stomach,STAD
best20GTEx_TCGA,0.704458,0.216465,258,83,38,45,0.971,0.447,0.495,0.902,0.539,0.437,0.821,0.458,0.895,0.605,0.189,0.934,0.713,0.702,0.276
best25GTEx_TCGA,0.709084,0.216436,322,83,44,43,0.988,0.440,0.508,0.906,0.539,0.456,0.804,0.516,0.895,0.612,0.180,0.934,0.717,0.698,0.271
best20FindAllDegree1_wl5,0.708614,0.216666,313,83,46,45,0.983,0.443,0.502,0.902,0.543,0.456,0.786,0.520,0.904,0.586,0.162,0.926,0.735,0.702,0.280



 best20FindAllDegree1_wl5 classes < 0.7 sensitivity


id
0                                       ACC
2                  Adipose_Visceral_Omentum
5                           Artery_Coronary
7                                      BLCA
8                                      BRCA
9                                   Bladder
10                           Brain_Amygdala
11     Brain_Anterior_cingulate_cortex_BA24
12              Brain_Caudate_basal_ganglia
16                 Brain_Frontal_Cortex_BA9
17                        Brain_Hippocampus
18                       Brain_Hypothalamus
19    Brain_Nucleus_accumbens_basal_ganglia
20              Brain_Putamen_basal_ganglia
23                    Breast_Mammary_Tissue
24                                     CESC
26                                     COAD
31                         Colon_Transverse
33                                     ESCA
34      Esophagus_Gastroesophageal_Junction
36                     Esophagus_Muscularis
38                                     HNSC
40                     Heart_

In [13]:
def compareDegree1Genes():
    # compare the degree1 genes
    b20DictPath = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best20GTEx_TCGA/training/best20GTEx_TCGA.sh.out/upsetPlot.out/best20.intersection.dict"
    b25DictPath = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best25GTEx_TCGA/training/best25GTEx_TCGA.sh.out/upsetPlot.out/best25.intersection.dict"
    b20FindAllPath = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best20FindAllDegree1_wl5/training/best20FindAllDegree1_wl5.sh.out/upsetPlot.out/best20_findAllDegree1_wl5.intersection.dict"

    print()
    b20Dict = loadDictionary(b20DictPath)    
    best20_D1_Dict = findIntersectionsWithDegree(b20Dict , degree=1)
    #print(f'best20_D1_Dict \n{pp.pformat(best20_D1_Dict)}')
    b20_D1_keySet = set(best20_D1_Dict.keys())
    #print(f'b20_D1_keySet \n{pp.pformat(set(b20_D1_keySet))}')

    print('\n###########')
    b25Dict = loadDictionary(b25DictPath)
    b25_D1_Dict= findIntersectionsWithDegree(b25Dict , degree=1)
    #print(f'b25_D1_Dict \n{pp.pformat(b25_D1_Dict)}')
    b25_D1_keySet = set(b25_D1_Dict.keys())    
    #print(f'b25_D1_keySet \n{pp.pformat(b25_D1_keySet)}')

    print('\n###########')
    b20FindAllDict = loadDictionary(b20FindAllPath)
    b20FindAll_D1_Dict= findIntersectionsWithDegree(b20FindAllDict , degree=1)
    #print(f'b20FindAll_D1_Dict \n{pp.pformat(b20FindAll_D1_Dict)}')
    b20FindAll_D1_keySet = set(b20FindAll_D1_Dict.keys())    
    #print(f'b20FindAll_D1_keys \n{pp.pformat(b20FindAll_D1_keys)}')

    print(f'keys: b20 - b25 {b20_D1_keySet - b25_D1_keySet}')
    print(f'keys: b25 - b20 {b25_D1_keySet - b20_D1_keySet}')

    print()
    print(f'keys: b20 - b20findAll {b20_D1_keySet - b20FindAll_D1_keySet}')
    print(f'keys: b20findAll - b20 {b20FindAll_D1_keySet - b20_D1_keySet}')

    print()
    print("???? how come b25 did not have {('LUSC',), ('THCA',)}) ????")
    print('I think when we add an additional best 5 genes the b20 LUSC and THCA where no longer unique')
    print('this suggestest that we will get better discriminators by starting with a larger topN\n')
    print(f'keys: b25_D1_keySet - b20FindAll_D1_keySet {b25_D1_keySet - b20FindAll_D1_keySet}')
    print(f'keys: b20FindAll_D1_keySet - b25_D1_keySet {b20FindAll_D1_keySet - b25_D1_keySet}')

    print()
    print(f"lung in b20FindAll_D1_keySet: {('Lung',) in b20FindAll_D1_keySet}")
    print(f"LUAD in b20FindAll_D1_keySet: {('LUAD',) in b20FindAll_D1_keySet}")
    print(f"LUSC in b20FindAll_D1_keySet: {('LUSC',) in b20FindAll_D1_keySet}")

compareDegree1Genes()



###########

###########
keys: b20 - b25 {('THCA',), ('LUSC',)}
keys: b25 - b20 {('Testis',), ('UCS',), ('LGG',), ('THYM',), ('Brain_Cerebellar_Hemisphere',), ('KIRP',), ('KIRC',), ('PRAD',)}

keys: b20 - b20findAll set()
keys: b20findAll - b20 {('Testis',), ('UCS',), ('LGG',), ('THYM',), ('Brain_Cerebellar_Hemisphere',), ('KIRP',), ('KIRC',), ('PRAD',)}

???? how come b25 did not have {('LUSC',), ('THCA',)}) ????
I think when we add an additional best 5 genes the b20 LUSC and THCA where no longer unique
this suggestest that we will get better discriminators by starting with a larger topN

keys: b25_D1_keySet - b20FindAll_D1_keySet set()
keys: b20FindAll_D1_keySet - b25_D1_keySet {('THCA',), ('LUSC',)}

lung in b20FindAll_D1_keySet: True
LUAD in b20FindAll_D1_keySet: True
LUSC in b20FindAll_D1_keySet: True


In [14]:
best500FindAllDegree1_wl500DictPath = "/private/groups/kimlab/aedavids/deconvolution/1vsAll-~gender_category/best500FindAllDegree1_wl500/training/best500FindAllDegree1_wl500.sh.out/upsetPlot.out/best500_findAllDegree1_wl500.intersection.dict"
best500FindAllDegree1_wl500Dict = loadDictionary(best500FindAllDegree1_wl500DictPath)
best500FindAllDegree1_wl500Degree1 = findIntersectionsWithDegree(best500FindAllDegree1_wl500Dict, degree=1)
degree1CategoriesSet = findAllCategories(best500FindAllDegree1_wl500Degree1)
degree1GenesSet = findAllGenes(best500FindAllDegree1_wl500Degree1)
print(f'len(degree1CategoriesSet) : {len(degree1CategoriesSet)}')
print(f'len(degree1GenesSet) : {len(degree1GenesSet)}')


len(degree1CategoriesSet) : 82
len(degree1GenesSet) : 3999


In [15]:
aedwip

NameError: name 'aedwip' is not defined

## stage aedwip: bestNDegree1GTEx_TCGA
This stage drops all genes that are not in an intersection with degree 1. These are signifigant in a single category. Given we have dropped a lot of genes we epxect pipeline stage to run faster. We want to use the fewest genes possible. ie. signatures are more likely to generalize. Any perfomance improved is probably do to reduction in confounding features.

<span style="color:red">Notice the log/print output reports</span>  
```
best200Degree1GTEx_TCGA types without degree 1 intersections: 
 set()
```

This is becuase these stages where missing degree 1 intersections for come classes. This is a bug. We need to ensure all categories have at least 1 degree1 gene before select only degree 1

In [ ]:
def degree1(threshold):
    '''
    wrapped to avoid global polution
    '''
    bestDegree1ResultsDirs = [    
            "best20Degree1GTEx_TCGA",     
            "best25Degree1GTEx_TCGA",     
            "best30Degree1GTEx_TCGA",     
            "best50Degree1GTEx_TCGA",     
            "best100Degree1GTEx_TCGA",     
            "best200Degree1GTEx_TCGA",  
            "best500Degree1GTEx_TCGA",  
    ]
    
    outFilePrefix = "bestDegree1"
    metric = 'sensitivity'
    degree1DF, degree1BellowThresholdDF = metricsRunner(root, outDir, outFilePrefix, bestDegree1ResultsDirs, 
                           metric=metric, threshold=threshold, verbose=False)
    
    display( degree1DF.loc[:, findSummaryMetricsCols('sensitivity') + elifeCols  ] )

    stageName =  "best200Degree1GTEx_TCGA"
    print(f'\n {stageName} classes < {threshold} {metric}')
    selectRowsDegree200Bellow = degree1BellowThresholdDF.loc[:, "stage"] == stageName
    display( degree1BellowThresholdDF.loc[selectRowsDegree200Bellow, 'category'] )
    
    stageName =  "best500Degree1GTEx_TCGA"
    print(f'\n {stageName} classes < {threshold} {metric}')
    selectRowsDegree1500Bellow = degree1BellowThresholdDF.loc[:, "stage"] == stageName
    
    display( degree1BellowThresholdDF.loc[selectRowsDegree1500Bellow, 'category'] )

degree1(threshold)

In [ ]:
aedwip

In [ ]:
def poorSensitivityAEDWIP():
    '''
    TODO parameterize
    '''
    metricCols = ['mean_sensitivity', 'std_sensitivity', 'numGenes', 'numTypes', 
                  'numDegree1', 'numAboveThreshold']
    
    selectSensitivityCols = ~bestUnique_6_DF.columns.isin( metricCols )
    sensitivityColsIndex = bestUnique_6_DF.columns[selectSensitivityCols]
    assert len(sensitivityColsIndex) == 83
    
    sensitivitySeries = bestUnique_6_DF.loc['best100Enriched_6_Degree1GTEx_TCGA', sensitivityColsIndex] 
    
    defaultThreshold = 0.7
    selectRows =  sensitivitySeries < defaultThreshold

    poorSensitivitySeries = sensitivitySeries.loc[selectRows] 
    
    return poorSensitivitySeries

bestUnique_6_PoorSensitivitySeries = poorSensitivityAEDWIP()
bestUnique_6_PoorSensitivitySeries

## check specificity

In [ ]:
def poorSpecificityAEDWIP(threshold):
    '''
    TODO parameterize
    '''
    metricCols = ['mean_specificity', 'std_specificity', 'numGenes', 'numTypes', 
                  'numDegree1', 'numAboveThreshold']
    
    specificityCols = ~bestUnique_6_SpecificityDF.columns.isin( metricCols )
    specificitySeries = bestUnique_6_SpecificityDF.loc['best100Enriched_6_Degree1GTEx_TCGA', specificityCols] 
 
    selectRows =  specificitySeries < threshold

    poorSpecificitySeries = specificitySeries.loc[selectRows] 
    
    return poorSpecificitySeries

threshold = 0.99
bestUnique_6_PoorSpecificitySeries = poorSpecificityAEDWIP(threshold=threshold)
print(f' categories with specificity < {threshold}' )
bestUnique_6_PoorSpecificitySeries

## Evaluate Elife performance on training data

In [ ]:
def evalMetrics(cols : list[str],
                sensitivityDF : pd.DataFrame,
                specificityDF : pd.DataFrame,
                runName : str,
               ) -> pd.DataFrame :
    '''
    TODO
    arguments:
        cols:
            example ["COAD", "READ", "Colon_Sigmoid", ]
    '''

    # sensitivitySeries = bestUnique_6_DF.loc['best100Enriched_6_Degree1GTEx_TCGA', cols]
    sensitivitySeries = sensitivityDF.loc[runName, cols]
    sensitivitySeries.name = "sensitivity"
    
    # specificitySeries = bestUnique_6_SpecificityDF.loc['best100Enriched_6_Degree1GTEx_TCGA', cols]
    specificitySeries = specificityDF.loc[runName, cols]
    specificitySeries.name = "specificity"
    
    byCols=1
    retDF = pd.concat( [sensitivitySeries, specificitySeries], axis=byCols).sort_values(by="id")

    return retDF

# Summary

TODO check and re-run enrichLUAD_LUSC_results

In [ ]:
summaryResultsDirs = [
        "best200GTEx_TCGA",
]

outFilePrefix = "summary"
summaryDF = metricsRunner(root, outDir, outFilePrefix, summaryResultsDirs, 
                       metric='sensitivity', threshold=threshold)

summaryDF.loc[:, findSummaryMetricsCols('sensitivity') + elifeCols  ]